In [47]:
import pandas as pd

from src.data import load_training_data, make_exog_features, split_data

In [48]:
data = load_training_data()
data.head()

,ba_AECI,ba_AVA,ba_AZPS,ba_BANC,ba_BPAT,ba_CHPD,ba_CISO,ba_CPLE,ba_CPLW,ba_DOPD,...,ba_SWPP,ba_TAL,ba_TEC,ba_TEPC,ba_TIDC,ba_TPWR,ba_TVA,ba_WACM,ba_WALC,ba_WAUW
datetime,,,,,,,,,,,,,,,,,,,,,
2022-10-01,51628.0,27070.0,96193.0,46398.0,123905.0,3745.0,592567.0,123640.0,11797.0,4617.0,...,649266.0,6601.0,49325.0,34960.0,8222.0,10804.0,348678.0,80299.0,24633.0,1820.0
2022-10-02,53127.0,28039.0,97208.0,42814.0,125317.0,3655.0,560074.0,124418.0,11759.0,4583.0,...,647709.0,6682.0,51702.0,37336.0,7194.0,10773.0,345900.0,79702.0,26100.0,1749.0
2022-10-03,54708.0,30110.0,96570.0,47041.0,133353.0,3790.0,623658.0,132803.0,12155.0,4732.0,...,689771.0,6886.0,54270.0,35662.0,8346.0,11228.0,373596.0,80536.0,25310.0,1915.0
2022-10-04,53345.0,30764.0,88963.0,48332.0,134664.0,3831.0,654561.0,134430.0,12294.0,4829.0,...,687579.0,6781.0,53101.0,31415.0,8765.0,10927.0,378071.0,80522.0,23938.0,1838.0
2022-10-05,53356.0,30421.0,91984.0,50362.0,135464.0,3815.0,664304.0,136348.0,12482.0,4854.0,...,687179.0,7009.0,54212.0,30993.0,8457.0,10962.0,376763.0,81362.0,23839.0,1828.0


In [49]:
data = make_exog_features(data)
data.filter(like="exog_").head()

,exog_is_holiday,exog_month,exog_day_of_week,exog_is_weekend,exog_season
datetime,,,,,
2022-10-01,0,10,5,1,4
2022-10-02,0,10,6,1,4
2022-10-03,0,10,0,0,4
2022-10-04,0,10,1,0,4
2022-10-05,0,10,2,0,4


### Build benchmark models

Build several benchmark models to compare model performance against. 

- Naive: previous day's demand as forecast. 
- Lag 7: same day a week ago as forecast. 
- Rolling mean 7: rolling weekly average as forecast. 
- Lag 30: use same day one month ago as forecast. 
- Rolling mean 30: rolling monthly average as forecast. 
- Lag 365: use same day one year ago as forecast. 
- Rolling mean 365: rolling annual average as forecast. 

In [50]:
benchmark = load_training_data()
target_columns = benchmark.filter(like="ba_").columns

In [51]:
from typing import List

def get_lag_columns(df: pd.DataFrame, lag: int, target_columns: List[str] = target_columns) -> pd.DataFrame:

    lag_columns = {
        f"lag{lag}_{col}".replace("ba_", ""): df[col].shift(periods=lag)
        for col in target_columns
    }

    # Convert the dictionary to a DataFrame
    lag_df = pd.DataFrame(lag_columns)

    # Concatenate the original and new DataFrame along the column axis
    df = pd.concat([df, lag_df], axis=1)

    return df

benchmark = get_lag_columns(benchmark, lag=1, target_columns=target_columns)
benchmark = get_lag_columns(benchmark, lag=7, target_columns=target_columns)
benchmark = get_lag_columns(benchmark, lag=30, target_columns=target_columns)
benchmark = get_lag_columns(benchmark, lag=365, target_columns=target_columns)

In [52]:
def get_rolling_average_columns(df: pd.DataFrame, window: int, target_columns: List[str] = target_columns) -> pd.DataFrame:

    for col in target_columns:
        result = (
            df[col]
            .rolling(window=window)  
            .agg(["mean"])  
            .shift(freq="1D")
        )

        result.columns = [f"rolling{window}_{col}".replace("ba_", "")]

        df = df.merge(result, how="left", left_index=True, right_index=True)
        
    return df

benchmark = get_rolling_average_columns(benchmark, window=7, target_columns=target_columns)
benchmark = get_rolling_average_columns(benchmark, window=30, target_columns=target_columns)
benchmark = get_rolling_average_columns(benchmark, window=365, target_columns=target_columns)

In [54]:
data_train, data_test = split_data(benchmark)

data_test.head()

data_train.shape=(396, 424)
data_test.shape=(366, 424)


,ba_AECI,ba_AVA,ba_AZPS,ba_BANC,ba_BPAT,ba_CHPD,ba_CISO,ba_CPLE,ba_CPLW,ba_DOPD,...,rolling365_SWPP,rolling365_TAL,rolling365_TEC,rolling365_TEPC,rolling365_TIDC,rolling365_TPWR,rolling365_TVA,rolling365_WACM,rolling365_WALC,rolling365_WAUW
datetime,,,,,,,,,,,,,,,,,,,,,
2023-11-01,74719.0,37869.0,75586.0,41192.0,163584.0,6291.0,581167.0,158292.0,15867.0,6911.0,...,772022.758904,7604.794521,61760.863014,38198.032877,7752.523288,13443.219178,435099.772603,89397.989041,23593.542466,2358.980822
2023-11-02,64962.0,36243.0,76424.0,41568.0,153571.0,5649.0,590289.0,175089.0,16396.0,6389.0,...,772322.446575,7603.460274,61720.369863,38210.539726,7752.731507,13444.786301,435334.295890,89398.895890,23558.005479,2359.487671
2023-11-03,58043.0,33648.0,75987.0,41414.0,145742.0,5585.0,580462.0,170746.0,14825.0,6244.0,...,772530.271233,7602.920548,61665.986301,38224.726027,7753.405479,13438.619178,435589.460274,89401.049315,23548.605479,2359.490411
2023-11-04,51220.0,32681.0,74389.0,39564.0,139123.0,5096.0,542774.0,145723.0,13370.0,5911.0,...,772606.052055,7600.235616,61627.515068,38236.991781,7754.038356,13429.183562,435734.616438,89377.547945,23542.827397,2358.958904
2023-11-05,54544.0,32567.0,79871.0,41281.0,143866.0,5399.0,549166.0,141021.0,12127.0,6120.0,...,772548.978082,7597.252055,61599.260274,38252.876712,7753.041096,13420.358904,435770.230137,89346.687671,23536.923288,2358.298630


We use the mean absolute error averaged over all series to evaluate each model. To do this, we need to filter for the model type, e.g. 'lag7', find the MAE for this type with respect to each series, then average the results. 

In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

all_means = []
for col in target_columns:
    # Getting the demand and benchmark scores for a given BA over the test period
    ba_string = col.replace("ba_", "")
    tmp = benchmark.filter(regex=f"{ba_string}$").loc[data_test.index].copy()
    ba = tmp.iloc[:, 0]
    maes = []
    # Calculating the MAE by comparing each score against the demand
    for c in tmp.columns[1:]:
        bm = tmp[c].copy()
        mae = mean_absolute_error(ba, bm)
        maes.append(mae)

    all_means.append(np.array(maes))

# Averaging each score over all of the BAs
stacked = np.stack(all_means)
final_means = np.mean(stacked, axis=0)

In [56]:
benchmark_names = [col.replace("_SC", "") for col in benchmark.filter(regex="SC$").columns[1:]]
pd.DataFrame({benchmark_names[i]: [final_means[i]] for i in range(len(benchmark_names))})

,lag1,lag7,lag30,lag365,rolling7,rolling30,rolling365
0,8926.185328,17554.469224,25404.884988,20028.418291,13141.285177,16141.537504,25568.907308


On this basis, using the previous day's demand as the forecast gives the smallest MAE, so we'll use this as a benchmark for comparison moving forward. 